In [1]:
import numpy as np
import cv2

import PIL.Image as Image
import os

import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [2]:
IMAGE_SHAPE = (224, 224)

# classifier = tf.keras.Sequential([
#     hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4", input_shape=IMAGE_SHAPE + (3,))
# ])

In [3]:
# gold_fish = Image.open('goldfish.jpg').resize(IMAGE_SHAPE)
# gold_fish

In [4]:
# gold_fish = np.array(gold_fish)/255.0
# gold_fish.shape

In [5]:
# gold_fish

In [6]:
# gold_fish[np.newaxis, ...].shape

In [7]:
# result = classifier.predict(gold_fish[np.newaxis, ...])
# result.shape

In [8]:
# predicted_label_index = np.argmax(result)
# predicted_label_index

In [9]:
# image_labels = []
# with open("ImageNetLabels.txt", "r") as f:
#     image_labels = f.read().splitlines()
# image_labels[:5]

In [10]:
# image_labels[predicted_label_index]

In [11]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url,  cache_dir='.', untar=True)

In [12]:
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

PosixPath('datasets/flower_photos')

In [13]:
# list(data_dir.glob('*/*.jpg'))[:5]

In [14]:
image_count = len(list(data_dir.glob('*/*.jpg')))
# print(image_count)

In [15]:
# roses = list(data_dir.glob('roses/*'))
# roses[:5]

In [16]:
# Image.open(str(roses[1]))

In [17]:
# tulips = list(data_dir.glob('tulips/*'))
# Image.open(str(tulips[0]))

In [18]:
flowers_images_dict = {
    'roses': list(data_dir.glob('roses/*')),
    'daisy': list(data_dir.glob('daisy/*')),
    'dandelion': list(data_dir.glob('dandelion/*')),
    'sunflowers': list(data_dir.glob('sunflowers/*')),
    'tulips': list(data_dir.glob('tulips/*')),
}

flowers_labels_dict = {
    'roses': 0,
    'daisy': 1,
    'dandelion': 2,
    'sunflowers': 3,
    'tulips': 4,
}

In [19]:
# flowers_images_dict['roses'][:5]

In [20]:
str(flowers_images_dict['roses'][0])

'datasets/flower_photos/roses/15424480096_45bb574b33.jpg'

In [21]:
img = cv2.imread(str(flowers_images_dict['roses'][0]))

In [22]:
# img.shape

In [23]:
# cv2.resize(img,(224,224)).shape

In [24]:
X, y = [], []

for flower_name, images in flowers_images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(224,224))
        X.append(resized_img)
        y.append(flowers_labels_dict[flower_name])

In [25]:
X = np.array(X)
y = np.array(y)

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [27]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [28]:
# X[0].shape

In [29]:
# IMAGE_SHAPE+(3,)

In [30]:
x0_resized = cv2.resize(X[0], IMAGE_SHAPE)
x1_resized = cv2.resize(X[1], IMAGE_SHAPE)
x2_resized = cv2.resize(X[2], IMAGE_SHAPE)

In [31]:
# plt.axis('off')
# plt.imshow(X[0])

In [32]:
# plt.axis('off')
# plt.imshow(X[1])

In [33]:
# plt.axis('off')
# plt.imshow(X[2])

In [34]:
# predicted = classifier.predict(np.array([x0_resized, x1_resized, x2_resized]))
# predicted = np.argmax(predicted, axis=1)
# predicted

In [35]:
# image_labels[795]

In [36]:
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224,224,3),trainable=False
)

In [37]:
num_of_flowers = 5

model = tf.keras.Sequential([
    pretrained_model_without_top_layer,
    tf.keras.layers.Dense(num_of_flowers)
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 5)                 6405      
                                                                 
Total params: 2264389 (8.64 MB)
Trainable params: 6405 (25.02 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


In [ ]:
model.compile(
  optimizer="adam",
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

model.fit(X_train_scaled, y_train, epochs=5)

In [ ]:
model.evaluate(X_test_scaled,y_test)